# Quadruped Navigation/Motion Control with Text Commands


In [1]:
import torch
import pybullet as p
import pybullet_data
import time
import math

In [2]:
PI = math.pi

Start Pybullet Client


In [3]:
physicsClient = p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setGravity(0, 0, -9.81)   # set gravity

# Load models
planeId = p.loadURDF('plane.urdf')

init_pos = [0, 0, 0.005]
init_orientation = p.getQuaternionFromEuler([0, 0, PI])
robotId = p.loadURDF('quadruped/quadruped.urdf', basePosition=init_pos, baseOrientation=init_orientation)

In [4]:
from scripts.lidar import lidar_sim

In [5]:
p.setJointMotorControl2(robotId, 1, controlMode=p.POSITION_CONTROL, targetPosition=PI/4)

In [6]:
lidar_idx = p.getNumJoints(robotId) - 1
num_rays = 45
ray_length = 10
time.sleep(2)

SIM_LENGTH = 1
for _ in range(SIM_LENGTH):
    
    data = lidar_sim(robotId, lidar_idx)
    
    p.stepSimulation()
    time.sleep(1/240)